# Packages to be installed

In [0]:
!pip install -U Flask
!pip install flask-ngrok


# Dependencies

In [0]:
import hashlib
import json
from flask import Flask, request
import requests
import time
from flask_ngrok import run_with_ngrok

# Create a single block

In [0]:
class blockTransaction():
  
  def __init__(self,index,transaction,timestamp,previousHash):
    self.index = index
    self.transaction = transaction
    self.timestamp = timestamp
    self.previousHash = previousHash

  def hashCompute(self):
    blockJson = json.dumps(self.__dict__,sort_keys=True)
    return hashlib.sha256(blockJson.encode()).hexdigest()




# Create the chain relationship with constraints

In [0]:
class blockChain():

  difficulty = 1

  def __init__(self): # Initialize with genesis block in chain, and a list of empty unconfirmed Tx
    self.unconfirmedTransactions = []
    self.chain = []
    self.createGenesis()

  def createGenesis(self):
    genesis = blockTransaction(0,[],time.time(),'0')
    genesis.hash = genesis.hashCompute() # Create a hash for the genesis block
    self.chain.append(genesis)

  def finalBlock(self): # Uncover details of the last block in chain
    return self.chain[-1]

  def proofOfWork(self,block):
    # Brute force recursive computation for POW
    block.nonce = 0
    computedHash = block.hashCompute()
    while not hashCompute.startswith('0'*blockChain.difficulty):
      block.nonce += 1
      computedHash = block.hashCompute()
    return computedHash

  def addBlock(self,block,proof): # Takes in the block and its hash
    previousHash = self.finalBlock.hash # Previous hash is the hash of last block in chain
    if previousHash != block.previousHash:
      return False
    if not blockChain.isValidProof(block,proof): # Ensure proof is valid in block (POW achieved)
      return False
    block.hash = proof # The hash of the newly confirmed block
    self.chain.append(block) # Confirm addition of block to the chain
    return True
  
  def isValidProof(self,block,block_hash):
    return ( block_hash.startswith('0'* blockChain.difficulty) and block_hash == block.hashCompute() )

  def newTransaction(self,transaction): # Add new Tx to the list of unconfirmed Tx
    self.unconfirmedTransactions.append(transaction)

  def mine(self):
    if not self.unconfirmedTransactions: # Ensure that there exists unconfirmed transactions
      return False
    last_block = self.finalBlock
    new_block = blockTransaction(index = last_block.index + 1,
                                 transaction = self.unconfirmedTransactions,
                                 timestamp = time.time(),
                                 previousHash = last_block.hash)
    proof = self.proofOfWork(new_block) # Recursive mining until POW is achieved, and hash of new block is generated
    self.addBlock(new_block, proof) # Adds new block with its generated hash
    self.unconfirmedTransactions = [] # Return empty list after confirming the transaction
    return new_block.index
  
  def checkChain(cls,chain): # iterates through entire chain to do a double check
    result = True
    previous_hash = '0' # Begin with Genesis block
    for block in chain:
      block_hash = block.hash # Store a copy of attribute before deleting it
      delattr(block,'hash') # Delete attribute
      if not cls.isValidProof(block,block.hash) or previous_hash != block.previousHash:
        result = False
        break
      block.hash = block_hash # After validating chain before, add backs attribute of the block
      previous_hash = block_hash # Check next block
      return result
